### How to Use treeShow_app
Run the treeShow_app function.
A window will open asking whether to load from JSON or read from a directory. Type 'load' or 'read'.
If 'load' is chosen, you'll be prompted to select a JSON file to load the tree structure.
If 'read' is chosen, you'll first select the directory, enter the file formats, and then choose a location to save the JSON file.
The application now fulfills the requirements: it allows the user to choose the operation, inputs the necessary paths through the GUI, and can save/load the tree data as JSON.

In [2]:
import os
import subprocess
import json
import math
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog
from tkinter import simpledialog
import threading

In [5]:
def count_files(directory, file_formats):
    """ Count files and folders with specified formats in a directory and its subdirectories.
    Files with no extension are counted if 'NOEXTENSION' is included in file_formats. """
    
    total_files = 0
    folder_count = 0
    has_matching_files = False
    include_no_extension = "NOEXTENSION" in file_formats

    for entry in os.listdir(directory):
        full_path = os.path.join(directory, entry)
        if os.path.isdir(full_path):
            subfolder_files, has_files_in_subfolder = count_files(full_path, file_formats)
            total_files += subfolder_files
            if has_files_in_subfolder:
                folder_count += 1
        else:
            # Check for files with specified extensions or no extension (if 'NOEXTENSION' is specified)
            if any(entry.lower().endswith(ext) for ext in file_formats if ext) or (include_no_extension and '.' not in entry):
                total_files += 1
                has_matching_files = True

    # Include the current directory in the folder count if it directly contains matching files
    if has_matching_files:
        folder_count += 1

    return total_files, folder_count

def populate_tree(tree, node, path, file_formats):
    """ Populate the tree with nodes for directories and collective file counts. """
    for entry in sorted(os.listdir(path)):
        full_path = os.path.join(path, entry)
        if os.path.isdir(full_path):
            file_count, folder_count = count_files(full_path, file_formats)
            if file_count > 0:
                display_text = f'{entry} ({file_count} files in {folder_count} folders)'
            else:
                display_text = f'{entry} (No matching files)'
            folder_id = tree.insert(node, 'end', text=display_text)
            populate_tree(tree, folder_id, full_path, file_formats)
            
def save_tree_data(tree, path):
    """ Save the tree data to a JSON file, adding .json extension if not already present. """
    # Ensure the path ends with .json
    if not path.lower().endswith('.json'):
        path += '.json'

    tree_data = []

    def collect_data(node):
        children = tree.get_children(node)
        if children:
            for child in children:
                item = tree.item(child)
                tree_data.append({'id': child, 'parent': node, 'text': item['text']})
                collect_data(child)

    collect_data('')  # Start from the root

    with open(path, 'w') as file:
        json.dump(tree_data, file)

    collect_data('')  # Start from the root
    with open(path, 'w') as file:
        json.dump(tree_data, file)

def load_tree_data(tree, path):
    """ Load the tree data from a JSON file. """
    with open(path, 'r') as file:
        tree_data = json.load(file)
    for item in tree_data:
        tree.insert(item['parent'], 'end', iid=item['id'], text=item['text'])

class SpinningCircle:
    def __init__(self, parent, x, y, radius):
        self.parent = parent
        self.x = x
        self.y = y
        self.radius = radius  # Increase the radius size
        self.angle = 0
        self.circle = None

        self.update()

    def draw_circle(self):
        start_x = self.x + self.radius * math.cos(self.angle)
        start_y = self.y + self.radius * math.sin(self.angle)
        end_x = self.x + self.radius * math.cos(self.angle + math.pi)
        end_y = self.y + self.radius * math.sin(self.angle + math.pi)

        if self.circle:
            self.parent.delete(self.circle)

        self.circle = self.parent.create_oval(start_x, start_y, end_x, end_y, fill='blue', outline='')

        # Draw the "Loading" text at the center of the circle
        self.parent.create_text(self.x, self.y, text="Loading", fill="black")

    def update(self):
        self.draw_circle()
        self.angle += 0.1
        self.parent.after(50, self.update)


In [8]:

def treeShow_app_E1():
    root = tk.Tk()
    root.title('TreeShow App')

    tree = ttk.Treeview(root)
    tree.pack(fill='both', expand=True)
    
    def start_process():
        choice = simpledialog.askstring("Input", "Type 'load' to load from JSON or 'read' to read from directory:")
        
        if choice.lower() == 'load':
            load_path = filedialog.askopenfilename(title='Select JSON File', filetypes=[('JSON Files', '*.json')])
            load_tree_data(tree, load_path)

        elif choice.lower() == 'read':
            directory_path = filedialog.askdirectory(title='Select Directory')
            if not directory_path:
                return

            file_formats = simpledialog.askstring("File Formats", "Enter file formats separated by comma, e.g., .dcm,.jpg \n you can find files with no extension by adding the exact term 'NOEXTENSION' in your list")
            if not file_formats:
                return
            
            save_path = filedialog.asksaveasfilename(title='Save as JSON', filetypes=[('JSON Files', '*.json')])
            if not save_path:
                return

            canvas = tk.Canvas(root, width=100, height=100)
            canvas.pack()
            spinning_circle = SpinningCircle(canvas, 50, 50, 40)

            def process_files():
                try:
                    populate_tree(tree, '', directory_path, file_formats)
                    save_tree_data(tree, save_path)
                finally:
                    canvas.destroy()

            threading.Thread(target=process_files).start()


    start_btn = tk.Button(root, text='Start', command=start_process)
    start_btn.pack()

    root.mainloop()
    
treeShow_app_E1()

Error opening folder: name 'tree' is not defined
Error opening folder: name 'tree' is not defined
Error opening folder: name 'tree' is not defined
Error opening folder: name 'tree' is not defined
Error opening folder: name 'tree' is not defined
